In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from backwardc import floodsz_backwardc2
from backwardc import norm2angle
from backwardc import angle2norm
from backwardc import normalization1
import matplotlib.tri as mtri
import matplotlib as mpl
import numpy as np
from scipy.spatial import Delaunay
from plotly.figure_factory import create_trisurf
import matplotlib as mpl
import joblib
%matplotlib widget

In [2]:
filename = 'dt21.joblib.pkl'
dt = joblib.load(filename)

In [3]:
dt

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=21,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [4]:
volume = 0.1

In [5]:
p = np.arange(-np.pi,np.pi+np.pi/90,np.pi/90)
t = np.arange(0,np.pi+np.pi/90,np.pi/90)
t2 = t-np.pi/2.0
P,T = np.meshgrid(p, t)
P2,T2 = np.meshgrid(p, t2)
mesh = np.array([P,T])
pt = mesh.T.reshape(-1, 2)
n = len(pt)
norm2d = np.empty([n,3])
cs2d = np.empty([n,3])
par2d = np.empty([n])
pt2d = np.empty([n,2])
for i in range(len(pt)):
    norm2d[i,:] = angle2norm(pt2d[i,:])
    cs2d[i,:],par2d[i]  = floodsz_backwardc2(norm2d[i,:],volume)
    cen = cs2d[i,:] - 0.5
    cen = cen / np.linalg.norm(cen)
    pt2d[i,:] = norm2angle(cen) 

In [6]:
le = 1000000
par3d = np.empty([le])
norm3d = np.zeros([le,3])
cs3d = np.empty([le,3])
pt3d = np.empty([le,2])
norm3d[:,0] = ( np.random.rand(le) - 0.5 ) * 2.0
norm3d[:,1] = ( np.random.rand(le) - 0.5 ) * 2.0
norm3d[:,2] = ( np.random.rand(le) - 0.5 ) * 2.0
for i in range(le):
    norm3d[i,:] = norm3d[i,:] / np.sum(np.abs(norm3d[i,:]))  
    cs3d[i,:],par3d[i]  = floodsz_backwardc2(norm3d[i,:],volume)
    pt3d[i,:] = norm2angle(norm3d[i,:]) 

In [7]:
xs2d = cs2d[:,0]
ys2d = cs2d[:,1]
zs2d = cs2d[:,2]
ph2d = pt2d[:,0]
th2d = pt2d[:,1]

In [8]:
xs3d = cs3d[:,0]
ys3d = cs3d[:,1]
zs3d = cs3d[:,2]
ph3d = pt3d[:,0]
th3d = pt3d[:,1]

In [55]:
tri = mtri.Triangulation(pt3d[:,0],pt3d[:,1])
n_tri = len(tri.triangles)
colors = np.zeros(n_tri)
for i in range(n_tri):
    colors[i] = float(int((par3d[tri.triangles[i,0]] + par3d[tri.triangles[i,1]] + par3d[tri.triangles[i,2]])/3.0))
    
norm = mpl.colors.Normalize(vmin=colors.min().min(), vmax=colors.max().max())

In [56]:
fig = create_trisurf(x=xs3d, y=ys3d, z=zs3d,color_func=colors,                         colormap="Portland",
                         simplices=tri.triangles,
                         plot_edges=False)
# fig = create_trisurf(x=norms[:,0], y=norms[:,1], z=norms[:,2]
fig.show()

In [11]:
plt.tricontourf(P.flatten(),T.flatten(),par2d)